In [1]:
# 이거 실행하고 다시 런타임 시작
# 한글 폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,266 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from pykrx import stock
sns.set(font='NaNumBarunGothic')


import warnings
warnings.filterwarnings('ignore')
import re
pd.options.display.max_columns = None

In [4]:
# 기준이 되는 재무제표 데이터 끌어오기(여기에 있는 종목만 선정하기 위해)
fs = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/real_real_df.csv')

# 종목코드 6자리로 채워주기
fs['Stock'] = fs['Stock'].astype('str')
fs['Stock'] = fs['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

stock_code = list(fs['Stock'])

In [5]:
# olhson o score를 도출하기 위해 필요한 kisvalue 데이터 불러오기 & 전처리
olhson_data = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/olhson_data.csv')


olhson_data.head(10)
def make_preprocessing(df):
  col_name = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    col_name.append(f"{df[i].iloc[5].split('/')[0]} ({df[i].iloc[4].split('/')[0]})")


  df.columns = col_name
  df.drop([0,1,2,3,4,5], axis=0, inplace=True)
  df =df.reset_index(drop=True)

  # 뒤에 널값인 데이터도 같이 들어왔는데 같이 삭제
  last_index = df[df['Name']=='(주)아바텍'].index # 데이터가 들어있는 맨 마지막 인덱스. 이 이후꺼를 삭제시키면 된다.
  df = df.iloc[:list(df[df['Name']=='(주)아바텍'].index).pop()+1, :]

  # stock code 없는 기업 삭제
  del_index = list(df[df['Stock'].isnull()].index)
  df.drop(del_index, axis=0, inplace = True)
  df = df.reset_index(drop= True)

  # 인수합병된 기업들 빼기
  df['인수합병여부'] = 0
  df['인수합병여부'] = df.apply(lambda x:1 if '인수목적' in x['Name'] else 0,axis=1)

  df = df[df['인수합병여부']==0]
  df.drop('인수합병여부', axis=1, inplace =True)
  df = df.reset_index(drop=True)




  # 문자열 float로 변경
  for i in df.columns[3:]:
    df[i] = df[i].astype('str')
    df[i] = df[i].apply(lambda x: re.sub(',', '',x))
    df[i] = df[i].astype('float')



  # null인 경우 0으로 채운다. -> o_score 도출을 위해서(널값이 많이 나오지는 않았다.)
  for i in df.columns:
    if '당기순이익' in i:
      df[i].fillna(0, inplace=True)
    if '유형,임대주택자산감가상각비' in i:
      df[i].fillna(0, inplace=True)
    if '무형자산상각비' in i:
      df[i].fillna(0, inplace=True)
    if '법인세비용 ' in i:
      df[i].fillna(0, inplace=True)

  return df

olhson = make_preprocessing(olhson_data)
olhson

,KIS,Stock,Name,자산총계 (2017),부채총계 (2017),유동자산(계) (2017),유동부채(계) (2017),당기순이익 (2017),자산총계 (2018),부채총계 (2018),유동자산(계) (2018),유동부채(계) (2018),당기순이익 (2018),자산총계 (2019),부채총계 (2019),유동자산(계) (2019),유동부채(계) (2019),당기순이익 (2019),자산총계 (2020),부채총계 (2020),유동자산(계) (2020),유동부채(계) (2020),당기순이익 (2020),자산총계 (2021),부채총계 (2021),유동자산(계) (2021),유동부채(계) (2021),당기순이익 (2021),영업이익(손실) (2017),영업이익(손실) (2018),영업이익(손실) (2019),영업이익(손실) (2020),영업이익(손실) (2021),"유형,임대주택자산감가상각비 (2017)",무형자산상각비 (2017),"유형,임대주택자산감가상각비 (2018)",무형자산상각비 (2018),"유형,임대주택자산감가상각비 (2019)",무형자산상각비 (2019),"유형,임대주택자산감가상각비 (2020)",무형자산상각비 (2020),"유형,임대주택자산감가상각비 (2021)",무형자산상각비 (2021),EBITDA (2017),EBITDA (2018),EBITDA (2019),EBITDA (2020),EBITDA (2021),법인세비용 (2017),법인세비용 (2018),법인세비용 (2019),법인세비용 (2020),법인세비용 (2021)
0,036644,389500,(주)에스비비테크,1.692558e+10,8.668831e+09,7.090299e+09,7.257253e+09,2.678790e+08,2.471722e+10,1.971819e+10,1.356880e+10,8.723921e+09,-1.917031e+09,1.948681e+10,1.927691e+10,8.293176e+09,3.591423e+09,-4.789125e+09,1.499052e+10,1.321184e+10,4.417464e+09,7.995986e+09,-3.276516e+09,1.763133e+10,1.103581e+10,7.302344e+09,6.321137e+09,-2.815924e+09,7.386570e+08,-1.088044e+09,-3.769930e+09,-2.616470e+09,-2.227144e+09,6.450680e+08,286697000.0,5.528850e+08,0.0,7.241710e+08,0.0,6.653120e+08,0.0,8.617540e+08,0.0,NaN,NaN,NaN,NaN,NaN,0.000000e+00,-1.451200e+07,-2.514400e+07,1.850100e+07,6.559000e+06
1,PM6283,441270,(주)파인엠텍,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.0,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,497730,296640,(주)이노룰스,8.788407e+09,6.710795e+09,4.828215e+09,5.665559e+09,7.684080e+08,1.598095e+10,9.435975e+09,6.745808e+09,7.579176e+09,1.770280e+08,1.578891e+10,7.108213e+09,7.552133e+09,4.520995e+09,6.186920e+08,1.300520e+10,5.923127e+09,7.703371e+09,5.275242e+09,1.591312e+09,1.753050e+10,4.482222e+09,1.128946e+10,3.720986e+09,3.009484e+09,1.500808e+09,4.809880e+08,2.768827e+09,1.187901e+09,3.314891e+09,0.000000e+00,0.0,8.444500e+07,2373000.0,1.511030e+08,3414000.0,1.968080e+08,8962000.0,2.180280e+08,15917000.0,NaN,NaN,NaN,NaN,NaN,0.000000e+00,-3.423000e+06,5.385820e+08,-6.886700e+07,2.891460e+08
3,499459,417970,모델솔루션(주),3.335852e+10,1.968374e+10,8.925932e+09,6.779529e+09,3.048896e+09,4.120729e+10,2.145321e+10,1.865476e+10,8.760334e+09,7.210181e+09,4.528936e+10,2.069394e+10,2.073613e+10,1.861937e+10,7.270528e+09,4.233815e+10,1.958681e+10,2.207469e+10,1.759698e+10,1.299209e+09,4.826561e+10,2.123166e+10,2.716175e+10,9.062293e+09,7.086006e+09,5.421588e+09,9.433205e+09,9.332657e+09,6.870288e+09,8.526295e+09,2.109577e+09,540555000.0,1.907083e+09,357272000.0,2.525531e+09,168237000.0,2.415000e+09,74824000.0,2.200679e+09,71012000.0,NaN,NaN,NaN,NaN,NaN,-4.620840e+08,3.790220e+08,-1.075930e+08,-7.888870e+08,1.121041e+09
4,HV2035,393890,더블유씨피(주),9.287483e+10,5.387944e+09,1.413538e+10,4.385492e+09,-1.886107e+09,1.577923e+11,1.056927e+10,7.660966e+09,6.746037e+09,-9.513513e+09,4.307295e+11,2.367912e+11,1.528209e+11,4.331020e+10,-6.513745e+09,5.231622e+11,3.598037e+11,1.238286e+11,1.279343e+11,-3.117833e+10,6.300173e+11,2.242597e+11,2.144150e+11,1.283054e+11,-9.963987e+09,-9.083040e+08,-1.092999e+10,-5.055078e+09,9.760562e+09,4.085575e+10,3.777000e+06,0.0,9.205490e+08,87535000.0,1.095077e+10,115370000.0,2.631548e+10,425461000.0,3.125809e+10,787955000.0,NaN,NaN,NaN,NaN,NaN,-7.330000e+06,2.040000e+05,-1.029000e+06,-8.520900e+07,2.749500e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,I73639,950110,SBI핀테크솔루션즈(주),3.053952e+10,1.122834e+10,1.303828e+10,6.999535e+09,0.000000e+00,7.522338e+10,4.086029e+10,1.317464e+10,2.285635e+10,0.000000e+00,7.549365e+10,3.986139e+10,1.51250

In [6]:
# fs 데이터에 있는 종목코드만 대상으로 진행하기
tmp = []
olhson_df = pd.DataFrame()
for i in olhson['Stock'].unique():
  if i in fs['Stock'].unique():
    idx = list(olhson[olhson['Stock']==i].index.values).pop()
    tmp.append(idx)
olhson_df  =pd.DataFrame(olhson, index = tmp).reset_index(drop=True)


In [7]:
olhson_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   KIS                    1279 non-null   object 
 1   Stock                  1279 non-null   object 
 2   Name                   1279 non-null   object 
 3   자산총계 (2017)            1279 non-null   float64
 4   부채총계 (2017)            1279 non-null   float64
 5   유동자산(계) (2017)         1279 non-null   float64
 6   유동부채(계) (2017)         1279 non-null   float64
 7   당기순이익 (2017)           1279 non-null   float64
 8   자산총계 (2018)            1279 non-null   float64
 9   부채총계 (2018)            1279 non-null   float64
 10  유동자산(계) (2018)         1279 non-null   float64
 11  유동부채(계) (2018)         1279 non-null   float64
 12  당기순이익 (2018)           1279 non-null   float64
 13  자산총계 (2019)            1279 non-null   float64
 14  부채총계 (2019)            1279 non-null   float64
 15  유동자산

In [8]:
# 2016년도 당기순이익 불러오기(Y score 구하기 위해)
olhson_data2 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/olhson_data2.csv')
olhson_data2 = olhson_data2.loc[:,:'Unnamed: 3']


def make_preprocessing2(df):
  col_name = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    col_name.append(f"{df[i].iloc[5].split('/')[0]} ({df[i].iloc[4].split('/')[0]})")


  df.columns = col_name
  df.drop([0,1,2,3,4,5], axis=0, inplace=True)
  df =df.reset_index(drop=True)

  # 뒤에 널값인 데이터도 같이 들어왔는데 같이 삭제
  last_index = df[df['Name']=='(주)아바텍'].index # 데이터가 들어있는 맨 마지막 인덱스. 이 이후꺼를 삭제시키면 된다.
  df = df.iloc[:list(df[df['Name']=='(주)아바텍'].index).pop()+1, :]

  # stock code 없는 기업 삭제
  del_index = list(df[df['Stock'].isnull()].index)
  df.drop(del_index, axis=0, inplace = True)
  df = df.reset_index(drop= True)

  # 인수합병된 기업들 빼기
  df['인수합병여부'] = 0
  df['인수합병여부'] = df.apply(lambda x:1 if '인수목적' in x['Name'] else 0,axis=1)

  df = df[df['인수합병여부']==0]
  df.drop('인수합병여부', axis=1, inplace =True)
  df = df.reset_index(drop=True)




  # 문자열 float로 변경
  for i in df.columns[3:]:
    df[i] = df[i].astype('str')
    df[i] = df[i].apply(lambda x: re.sub(',', '',x))
    df[i] = df[i].astype('float')



  # null인 경우 0으로 채운다. -> o_score 도출을 위해서(널값이 많이 나오지는 않았다.)
  for i in df.columns:
    if '당기순이익' in i:
      df[i].fillna(0, inplace=True)

  return df


olhson_2016_net_loss = make_preprocessing2(olhson_data2)

# fs 데이터에 있는 종목코드만 대상으로 진행하기
tmp = []
olhson_df2 = pd.DataFrame()
for i in olhson_2016_net_loss['Stock'].unique():
  if i in fs['Stock'].unique():
    idx = list(olhson_2016_net_loss[olhson_2016_net_loss['Stock']==i].index.values).pop()
    tmp.append(idx)
olhson_df2  =pd.DataFrame(olhson_2016_net_loss, index = tmp).reset_index(drop=True)

olhson_df2

,KIS,Stock,Name,당기순이익(손실) (2016)
0,036644,389500,(주)에스비비테크,1.134724e+09
1,497730,296640,(주)이노룰스,5.368640e+08
2,499459,417970,모델솔루션(주),4.804671e+09
3,HS8995,314140,(주)알피바이오,1.569932e+09
4,867055,129920,(주)대성하이텍,2.224502e+09
...,...,...,...,...
1274,I73639,950110,SBI핀테크솔루션즈(주),1.129726e+09
1275,125028,046970,(주)우리로,-3.048900e+07
1276,383046,113810,디젠스(주),1.652004e+09
1277,F11411,155650,와이엠씨(주),8.297183e+09


In [9]:
# 2016년 당기순이익 추가해주기
olhson_df['당기순이익 (2016)'] = olhson_df2['당기순이익(손실) (2016)']
olhson_df.head(5)

,KIS,Stock,Name,자산총계 (2017),부채총계 (2017),유동자산(계) (2017),유동부채(계) (2017),당기순이익 (2017),자산총계 (2018),부채총계 (2018),유동자산(계) (2018),유동부채(계) (2018),당기순이익 (2018),자산총계 (2019),부채총계 (2019),유동자산(계) (2019),유동부채(계) (2019),당기순이익 (2019),자산총계 (2020),부채총계 (2020),유동자산(계) (2020),유동부채(계) (2020),당기순이익 (2020),자산총계 (2021),부채총계 (2021),유동자산(계) (2021),유동부채(계) (2021),당기순이익 (2021),영업이익(손실) (2017),영업이익(손실) (2018),영업이익(손실) (2019),영업이익(손실) (2020),영업이익(손실) (2021),"유형,임대주택자산감가상각비 (2017)",무형자산상각비 (2017),"유형,임대주택자산감가상각비 (2018)",무형자산상각비 (2018),"유형,임대주택자산감가상각비 (2019)",무형자산상각비 (2019),"유형,임대주택자산감가상각비 (2020)",무형자산상각비 (2020),"유형,임대주택자산감가상각비 (2021)",무형자산상각비 (2021),EBITDA (2017),EBITDA (2018),EBITDA (2019),EBITDA (2020),EBITDA (2021),법인세비용 (2017),법인세비용 (2018),법인세비용 (2019),법인세비용 (2020),법인세비용 (2021),당기순이익 (2016)
0,036644,389500,(주)에스비비테크,1.692558e+10,8.668831e+09,7.090299e+09,7.257253e+09,2.678790e+08,2.471722e+10,1.971819e+10,1.356880e+10,8.723921e+09,-1.917031e+09,1.948681e+10,1.927691e+10,8.293176e+09,3.591423e+09,-4.789125e+09,1.499052e+10,1.321184e+10,4.417464e+09,7.995986e+09,-3.276516e+09,1.763133e+10,1.103581e+10,7.302344e+09,6.321137e+09,-2.815924e+09,7.386570e+08,-1.088044e+09,-3.769930e+09,-2.616470e+09,-2.227144e+09,6.450680e+08,286697000.0,5.528850e+08,0.0,7.241710e+08,0.0,6.653120e+08,0.0,8.617540e+08,0.0,NaN,NaN,NaN,NaN,NaN,0.0,-14512000.0,-25144000.0,18501000.0,6.559000e+06,1.134724e+09
1,497730,296640,(주)이노룰스,8.788407e+09,6.710795e+09,4.828215e+09,5.665559e+09,7.684080e+08,1.598095e+10,9.435975e+09,6.745808e+09,7.579176e+09,1.770280e+08,1.578891e+10,7.108213e+09,7.552133e+09,4.520995e+09,6.186920e+08,1.300520e+10,5.923127e+09,7.703371e+09,5.275242e+09,1.591312e+09,1.753050e+10,4.482222e+09,1.128946e+10,3.720986e+09,3.009484e+09,1.500808e+09,4.809880e+08,2.768827e+09,1.187901e+09,3.314891e+09,0.000000e+00,0.0,8.444500e+07,2373000.0,1.511030e+08,3414000.0,1.968080e+08,8962000.0,2.180280e+08,15917000.0,NaN,NaN,NaN,NaN,NaN,0.0,-3423000.0,538582000.0,-68867000.0,2.891460e+08,5.368640e+08
2,499459,417970,모델솔루션(주),3.335852e+10,1.968374e+10,8.925932e+09,6.779529e+09,3.048896e+09,4.120729e+10,2.145321e+10,1.865476e+10,8.760334e+09,7.210181e+09,4.528936e+10,2.069394e+10,2.073613e+10,1.861937e+10,7.270528e+09,4.233815e+10,1.958681e+10,2.207469e+10,1.759698e+10,1.299209e+09,4.826561e+10,2.123166e+10,2.716175e+10,9.062293e+09,7.086006e+09,5.421588e+09,9.433205e+09,9.332657e+09,6.870288e+09,8.526295e+09,2.109577e+09,540555000.0,1.907083e+09,357272000.0,2.525531e+09,168237000.0,2.415000e+09,74824000.0,2.200679e+09,71012000.0,NaN,NaN,NaN,NaN,NaN,-462084000.0,379022000.0,-107593000.0,-788887000.0,1.121041e+09,4.804671e+09
3,HS8995,314140,(주)알피바이오,6.457702e+10,2.455539e+10,3.334482e+10,1.454381e+10,5.881569e+09,8.959963e+10,3.689531e+10,4.295203e+10,3.000506e+10,4.363446e+09,1.159244e+11,5.919584e+10,3.311131e+10,3.332302e+10,3.423335e+09,1.304471e+11,6.941174e+10,4.319151e+10,6.830001e+10,4.615509e+09,1.437370e+11,7.881001e+10,5.227193e+10,7.545094e+10,3.812561e+09,6.750606e+09,6.382369e+09,3.959920e+09,4.157063e+09,5.879847e+09,2.148062e+09,24708000.0,1.866447e+09,84493000.0,1.824337e+09,105736000.0,4.233249e+09,181106000.0,4.988008e+09,215446000.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-152899000.0,3.559160e+08,1.569932e+09
4,867055,129920,(주)대성하이텍,1.143085e+11,7.701378e+10,7.778543e+10,4.514462e+10,2.296441e+09,1.238696e+11,7.676503e+10,7.966123e+10,4.880179e+10,1.865670e+09,1.264083e+11,7.503042e+10,8.274118e+10,4.780276e+10,3.099913e+09,1.182687e+11,8.670698e+10,6.825128e+10,5.116868e+10,-1.173379e+10,1.427651e+11,1.117834e+11,8.730808e+10,8.915013e+10,7.457426e+09,6.727917e+09,4.717809e+09,5.169880e+09,-8.452512e+09,1.169168e+10,1.987212e+09,632705000.0,1.764138e+09,82991000.0,1.473634e+09,188379000.0,1.739449e+09,296903000.0,2.745202e+09,225291000.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.760807e+09,2.224502e+09


In [10]:
olhson_df['당기순이익 (2016)'].isna().sum()

0

In [11]:
def olhson_o_score(df):
  year = ['2018', '2019', '2020', '2021']
  # df['GDP_2018'] = 1898192.6e+9
  # df['GDP_2019'] = 1924498.1e+9
  # df['GDP_2020'] = 1940726.2e+9
  # df['GDP_2021'] = 2071658.0e+9
  df['GDP_2018'] = 113.0
  df['GDP_2019'] = 112.0 
  df['GDP_2020'] = 113.8 
  df['GDP_2021'] = 116.6
  for y in year:

    for col in df.columns:
      if y in col:
        if '자산총계' in col:
          자산총계 = col
        if '부채총계' in col:
          부채총계 = col
        if '유동자산' in col:
          유동자산 = col
        if '유동부채' in col:
          유동부채 = col
        if  '당기순이익' in col:
          당기순이익 = col
        if '법인세비용' in col:
          법인세비용 = col
        if '유형,임대주택자산감가상각비' in col:
          유형_임대주택자산감가상각비 = col
        if '무형자산상각비' in col:
          무형자산상각비 = col
      elif str(int(y)-1) in col: # 전년도 데이터 얻어야하는 것만
        if '당기순이익' in col:
          전년당기순이익 = col
      elif str(int(y)-2) in col: # 전전년도 데이터 얻어야 한다.
        if '당기순이익' in col:
          전전년당기순이익 = col


    df[f'{y}_X'] = df.apply(lambda x:1 if x[f'{부채총계}']>x[f'{자산총계}'] else 0,axis=1)
    df[f'{y}_Y'] = df.apply(lambda x:1 if x[f'{전년당기순이익}']<0 and x[f'{전전년당기순이익}']<0 else 0, axis=1)
    df[f'{y}_tmp'] = (df[f'{당기순이익}'] - df[f'{전년당기순이익}']) / (df[f'{당기순이익}'].abs() + df[f'{전년당기순이익}'].abs()) # nan값인 것을 0으로 채우기 위해
    df[f'{y}_tmp'].fillna(0, inplace=True)
    
    df[f'{y}_O_score'] = -1.32 - 0.407 * np.log(df[f'{자산총계}'] /  df[f'GDP_{y}']) + 6.03 * (df[f'{부채총계}'] / df[f'{자산총계}']) - 1.43 * ((df[f'{유동자산}'] - df[f'{유동부채}']) / df[f'{자산총계}']) + 0.0757 * (df[f'{유동부채}'] / df[f'{유동자산}']) -1.72 * df[f'{y}_X']  -2.37 * (df[f'{당기순이익}'] / df[f'{자산총계}']) -1.83 * (df[f'{당기순이익}'] + df[f'{법인세비용}'] + df[f'{유형_임대주택자산감가상각비}'] + df[f'{무형자산상각비}']) / df[f'{부채총계}'] + 0.285 * df[f'{y}_Y'] - 0.521 * (df[f'{y}_tmp'])

  return df

In [12]:
final_olhson = olhson_o_score(olhson_df)
final_olhson

,KIS,Stock,Name,자산총계 (2017),부채총계 (2017),유동자산(계) (2017),유동부채(계) (2017),당기순이익 (2017),자산총계 (2018),부채총계 (2018),유동자산(계) (2018),유동부채(계) (2018),당기순이익 (2018),자산총계 (2019),부채총계 (2019),유동자산(계) (2019),유동부채(계) (2019),당기순이익 (2019),자산총계 (2020),부채총계 (2020),유동자산(계) (2020),유동부채(계) (2020),당기순이익 (2020),자산총계 (2021),부채총계 (2021),유동자산(계) (2021),유동부채(계) (2021),당기순이익 (2021),영업이익(손실) (2017),영업이익(손실) (2018),영업이익(손실) (2019),영업이익(손실) (2020),영업이익(손실) (2021),"유형,임대주택자산감가상각비 (2017)",무형자산상각비 (2017),"유형,임대주택자산감가상각비 (2018)",무형자산상각비 (2018),"유형,임대주택자산감가상각비 (2019)",무형자산상각비 (2019),"유형,임대주택자산감가상각비 (2020)",무형자산상각비 (2020),"유형,임대주택자산감가상각비 (2021)",무형자산상각비 (2021),EBITDA (2017),EBITDA (2018),EBITDA (2019),EBITDA (2020),EBITDA (2021),법인세비용 (2017),법인세비용 (2018),법인세비용 (2019),법인세비용 (2020),법인세비용 (2021),당기순이익 (2016),GDP_2018,GDP_2019,GDP_2020,GDP_2021,2018_X,2018_Y,2018_tmp,2018_O_score,2019_X,2019_Y,2019_tmp,2019_O_score,2020_X,2020_Y,2020_tmp,2020_O_score,2021_X,2021_Y,2021_tmp,2021_O_score
0,036644,389500,(주)에스비비테크,1.692558e+10,8.668831e+09,7.090299e+09,7.257253e+09,2.678790e+08,2.471722e+10,1.971819e+10,1.356880e+10,8.723921e+09,-1.917031e+09,1.948681e+10,1.927691e+10,8.293176e+09,3.591423e+09,-4.789125e+09,1.499052e+10,1.321184e+10,4.417464e+09,7.995986e+09,-3.276516e+09,1.763133e+10,1.103581e+10,7.302344e+09,6.321137e+09,-2.815924e+09,7.386570e+08,-1.088044e+09,-3.769930e+09,-2.616470e+09,-2.227144e+09,6.450680e+08,286697000.0,5.528850e+08,0.0,7.241710e+08,0.0,6.653120e+08,0.0,8.617540e+08,0.0,NaN,NaN,NaN,NaN,NaN,0.000000e+00,-1.451200e+07,-2.514400e+07,1.850100e+07,6.559000e+06,1.134724e+09,113.0,112.0,113.8,116.6,0,0,-1.000000,-3.724200,0,0,-0.428277,-2.195952,0,1,0.187537,-2.072031,0,1,0.075601,-4.278181
1,497730,296640,(주)이노룰스,8.788407e+09,6.710795e+09,4.828215e+09,5.665559e+09,7.684080e+08,1.598095e+10,9.435975e+09,6.745808e+09,7.579176e+09,1.770280e+08,1.578891e+10,7.108213e+09,7.552133e+09,4.520995e+09,6.186920e+08,1.300520e+10,5.923127e+09,7.703371e+09,5.275242e+09,1.591312e+09,1.753050e+10,4.482222e+09,1.128946e+10,3.720986e+09,3.009484e+09,1.500808e+09,4.809880e+08,2.768827e+09,1.187901e+09,3.314891e+09,0.000000e+00,0.0,8.444500e+07,2373000.0,1.511030e+08,3414000.0,1.968080e+08,8962000.0,2.180280e+08,15917000.0,NaN,NaN,NaN,NaN,NaN,0.000000e+00,-3.423000e+06,5.385820e+08,-6.886700e+07,2.891460e+08,5.368640e+08,113.0,112.0,113.8,116.6,0,0,-0.625510,-4.989105,0,0,0.555050,-7.191237,0,0,0.440099,-7.393606,0,0,0.308245,-10.043585
2,499459,417970,모델솔루션(주),3.335852e+10,1.968374e+10,8.925932e+09,6.779529e+09,3.048896e+09,4.120729e+10,2.145321e+10,1.865476e+10,8.760334e+09,7.210181e+09,4.528936e+10,2.069394e+10,2.073613e+10,1.861937e+10,7.270528e+09,4.233815e+10,1.958681e+10,2.207469e+10,1.759698e+10,1.299209e+09,4.826561e+10,2.123166e+10,2.716175e+10,9.062293e+09,7.086006e+09,5.421588e+09,9.433205e+09,9.332657e+09,6.870288e+09,8.526295e+09,2.109577e+09,540555000.0,1.907083e+09,357272000.0,2.525531e+09,168237000.0,2.415000e+09,74824000.0,2.200679e+09,71012000.0,NaN,NaN,NaN,NaN,NaN,-4.620840e+08,3.790220e+08,-1.075930e+08,-7.888870e+08,1.121041e+09,4.804671e+09,113.0,112.0,113.8,116.6,0,0,0.405620,-7.978835,0,0,0.004167,-7.883736,0,0,-0.696791,-6.643196,0,0,0.690119,-8.864505
3,HS8995,314140,(주)알피바이오,6.457702e+10,2.455539e+10,3.334482e+10,1.454381e+10,5.881569e+09,8.959963e+10,3.689531e+10,4.295203e+10,3.000506e+10,4.363446e+09,1.159244e+11,5.919584e+10,3.311131e+10,3.332302e+10,3.423335e+09,1.304471e+11,6.941174e+10,4.319151e+10,6.830001e+10,4.615509e+09,1.437370e+11,7.881001e+10,5.227193e+10,7.545094e+10,3.812561e+09,6.750606e+09,6.382369e+09,3.959920e+09,4.157063e+09,5.879847e+09,2.148062e+09,24708000.0,1.866447e+09,84493000.0,1.824337e+09,105736000.0,4.233249e+09,181106000.0,4.988008e+09,215446000.0,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,-1.528990e+08,3.559160e+08,1.569932e+09,113.0,112.0,113.8,116.6,0,0,-0.148182,-7.682056,0,0,-0.120732,-6.783006,0,0,0.148302,-6.601536,0,0,-0.095271,-6.424298
4,867055,129920,(주)대성하이텍,1.143085e+11,7.701378e+10,7.7785

In [13]:
final_olhson.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 74 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   KIS                    1279 non-null   object 
 1   Stock                  1279 non-null   object 
 2   Name                   1279 non-null   object 
 3   자산총계 (2017)            1279 non-null   float64
 4   부채총계 (2017)            1279 non-null   float64
 5   유동자산(계) (2017)         1279 non-null   float64
 6   유동부채(계) (2017)         1279 non-null   float64
 7   당기순이익 (2017)           1279 non-null   float64
 8   자산총계 (2018)            1279 non-null   float64
 9   부채총계 (2018)            1279 non-null   float64
 10  유동자산(계) (2018)         1279 non-null   float64
 11  유동부채(계) (2018)         1279 non-null   float64
 12  당기순이익 (2018)           1279 non-null   float64
 13  자산총계 (2019)            1279 non-null   float64
 14  부채총계 (2019)            1279 non-null   float64
 15  유동자산

In [14]:
final_olhson[['2018_O_score', '2019_O_score', '2020_O_score', '2021_O_score']].describe()

,2018_O_score,2019_O_score,2020_O_score,2021_O_score
count,1279.000000,1279.000000,1279.000000,1279.000000
mean,-8.005972,-7.971533,-8.231908,-8.561288
std,3.055916,2.958501,3.169158,2.583866
min,-21.154586,-18.023054,-72.963103,-23.070736
25%,-9.732681,-9.754760,-9.819606,-10.081665
50%,-7.930918,-7.965731,-8.078489,-8.412924
75%,-6.435040,-6.422934,-6.493281,-7.006976
max,20.107096,18.863445,7.713168,4.069265


In [15]:
# 0.5 이상이면 2년이내 부도할 확률이 높은 기업
final_ohlson = final_olhson[['Stock', '2018_O_score', '2019_O_score', '2020_O_score', '2021_O_score']]

# 부도 확률
from math import exp
final_ohlson['2018_부도확률'] = final_ohlson.apply(lambda x: exp(x['2018_O_score']) / (1+exp(x['2018_O_score'])), axis=1)
final_ohlson['2019_부도확률'] = final_ohlson.apply(lambda x: exp(x['2019_O_score']) / (1+exp(x['2019_O_score'])), axis=1)
final_ohlson['2020_부도확률'] = final_ohlson.apply(lambda x: exp(x['2020_O_score']) / (1+exp(x['2020_O_score'])), axis=1)
final_ohlson['2021_부도확률'] = final_ohlson.apply(lambda x: exp(x['2021_O_score']) / (1+exp(x['2021_O_score'])), axis=1)

final_ohlson['2018_부도위험기업'] = final_ohlson.apply(lambda x: 1 if x['2018_O_score']>0.5 else 0, axis=1)
final_ohlson['2019_부도위험기업'] = final_ohlson.apply(lambda x: 1 if x['2019_O_score']>0.5 else 0, axis=1)
final_ohlson['2020_부도위험기업'] = final_ohlson.apply(lambda x: 1 if x['2020_O_score']>0.5 else 0, axis=1)
final_ohlson['2021_부도위험기업'] = final_ohlson.apply(lambda x: 1 if x['2021_O_score']>0.5 else 0, axis=1)

final_ohlson

,Stock,2018_O_score,2019_O_score,2020_O_score,2021_O_score,2018_부도확률,2019_부도확률,2020_부도확률,2021_부도확률,2018_부도위험기업,2019_부도위험기업,2020_부도위험기업,2021_부도위험기업
0,389500,-3.724200,-2.195952,-2.072031,-4.278181,0.023564,0.100115,0.111845,0.013678,0,0,0,0
1,296640,-4.989105,-7.191237,-7.393606,-10.043585,0.006766,0.000753,0.000615,0.000043,0,0,0,0
2,417970,-7.978835,-7.883736,-6.643196,-8.864505,0.000343,0.000377,0.001301,0.000141,0,0,0,0
3,314140,-7.682056,-6.783006,-6.601536,-6.424298,0.000461,0.001132,0.001356,0.001619,0,0,0,0
4,129920,-6.434970,-6.879751,-4.538230,-5.863938,0.001602,0.001027,0.010579,0.002832,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,950110,-5.944313,-6.149753,-5.730077,-5.203737,0.002614,0.002129,0.003236,0.005466,0,0,0,0
1275,046970,-6.483728,-4.189711,-3.719468,-9.892794,0.001526,0.014925,0.023673,0.000051,0,0,0,0
1276,113810,-4.753323,-6.297839,-4.321420,-5.912439,0.008549,0.001837,0.013107,0.002698,0,0,0,0
1277,155650,-7.635841,-7.428764,-10.620891,-10.415135,0.000483,0.000594,0.000024,0.000030,0,0,0,0


In [16]:
print(final_ohlson['2018_부도위험기업'].value_counts())
print(final_ohlson['2019_부도위험기업'].value_counts())
print(final_ohlson['2020_부도위험기업'].value_counts())
print(final_ohlson['2021_부도위험기업'].value_counts())

0    1269
1      10
Name: 2018_부도위험기업, dtype: int64
0    1265
1      14
Name: 2019_부도위험기업, dtype: int64
0    1275
1       4
Name: 2020_부도위험기업, dtype: int64
0    1274
1       5
Name: 2021_부도위험기업, dtype: int64


In [18]:
final_ohlson.to_csv('ohlson.csv', index=False)

In [17]:
final_ohlson

,Stock,2018_O_score,2019_O_score,2020_O_score,2021_O_score,2018_부도확률,2019_부도확률,2020_부도확률,2021_부도확률,2018_부도위험기업,2019_부도위험기업,2020_부도위험기업,2021_부도위험기업
0,389500,-3.724200,-2.195952,-2.072031,-4.278181,0.023564,0.100115,0.111845,0.013678,0,0,0,0
1,296640,-4.989105,-7.191237,-7.393606,-10.043585,0.006766,0.000753,0.000615,0.000043,0,0,0,0
2,417970,-7.978835,-7.883736,-6.643196,-8.864505,0.000343,0.000377,0.001301,0.000141,0,0,0,0
3,314140,-7.682056,-6.783006,-6.601536,-6.424298,0.000461,0.001132,0.001356,0.001619,0,0,0,0
4,129920,-6.434970,-6.879751,-4.538230,-5.863938,0.001602,0.001027,0.010579,0.002832,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,950110,-5.944313,-6.149753,-5.730077,-5.203737,0.002614,0.002129,0.003236,0.005466,0,0,0,0
1275,046970,-6.483728,-4.189711,-3.719468,-9.892794,0.001526,0.014925,0.023673,0.000051,0,0,0,0
1276,113810,-4.753323,-6.297839,-4.321420,-5.912439,0.008549,0.001837,0.013107,0.002698,0,0,0,0
1277,155650,-7.635841,-7.428764,-10.620891,-10.415135,0.000483,0.000594,0.000024,0.000030,0,0,0,0
